In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
from pandas import *
import csv 
import warnings as warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tools.sm_exceptions import ConvergenceWarning # to ignore ConvergenceWarning


from tabulate import tabulate

In [2]:
#READING CSV FILES AND CREATING TRAINING/TEST DATA SETS

def num_rows_and_cols(filename):
    cur_file = open(filename,newline='')
    raw_data = csv.reader(cur_file)
    num_rows = 0 
    num_cols = 0 
    is_num_cols_known = 0
    for rows in raw_data:
        if is_num_cols_known == 0:
            for element in rows:
                num_cols += 1
            is_num_cols_known = 1 
        num_rows += 1
    cur_file.close()
    return num_rows,num_cols


def read_file(filename):
    Y = []
    X = []
    Y_training = [] 
    Y_test = []
    X_training = [] 
    X_test = []
    num_rows,num_columns = num_rows_and_cols(filename)
    num_cities = num_columns-1 
    for i in range(1,num_cities+1): 
        empty_list = [];
        X.append(empty_list)
    with open(filename,newline='') as cur_file: 
        raw_data = csv.reader(cur_file)
        for rows in raw_data:
            Y.append(float(rows[0]))
            for i in range(1,num_cities+1):
                X[i-1].append(float(rows[i]))
    return X,Y



X,Y = read_file("data1.csv")
x1 = X
X,Y = read_file("data2.csv")
x2 = X
X,Y = read_file("data3.csv")
x3 = X
x_combined = np.vstack((x1,x2,x3))
year = Y

Y_training = []
Y_test = []
for i in range(0, 10):
    Y_test.append(float(Y[i]))
for i in range(10, 30):
    Y_training.append(float(Y[i]))

def x_sets(z):
    X_training = []
    X_test = []
    X = x_combined[z]
    for i in range(0, 10):
        X_test.append(float(X[i]))
    for i in range(10, 30):
        X_training.append(float(X[i]))
    return X_training, X_test
    

X_training, X_test = x_sets(0)
alameda_training = X_training
alameda_test = X_test
X_training, X_test = x_sets(1)
sanmateo_training = X_training
sanmateo_test = X_test
X_training, X_test = x_sets(2)
losangeles_training = X_training
losangeles_test = X_test
X_training, X_test = x_sets(3)
merced_training = X_training
merced_test = X_test
X_training, X_test = x_sets(4)
santaclara_training = X_training
santaclara_test = X_test
X_training, X_test = x_sets(5)
riverside_training = X_training
riverside_test = X_test
X_training, X_test = x_sets(6)
sandiego_training = X_training
sandiego_test = X_test
X_training, X_test = x_sets(7)
santabarbara_training = X_training
santabarbara_test = X_test
X_training, X_test = x_sets(8)
santacruz_training = X_training
santacruz_test = X_test

x_training = np.vstack((alameda_training,sanmateo_training,losangeles_training,merced_training,santaclara_training,riverside_training,sandiego_training,santabarbara_training,santacruz_training))
# generate test stacked vectors
x_test = np.vstack((alameda_test,sanmateo_test,losangeles_test,merced_test,santaclara_test,riverside_test,sandiego_test,santabarbara_test,santacruz_test))

In [3]:
def ARIMA_val_loss(ytrain, ytest, p, d, q):
    model = ARIMA(ytrain,order=(p,d,q))
    try:
        model_fit = model.fit()
        y_pred = model_fit.predict(len(ytrain),len(ytrain)+len(ytest)-1)
        val_loss = np.sum((y_pred - ytest)**2)
        return val_loss
    except:
        return []

def polynomial_regression(xtrain,ytrain,degree):
    x = np.array(xtrain)
    y = np.array(ytrain)
    coeff = np.polyfit(x,y,degree)
    return coeff

# this function takes training and test data, and returns optimal loss alongside coefficients of best-fit polynomial
def validation_loss(xtrain,ytrain,xtest,ytest): 
    max_degree = len(xtrain)-1
    for deg in range(max_degree+1):
        cur_coeff = polynomial_regression(xtrain,ytrain,deg)
        y_pred = np.polyval(cur_coeff,xtest)
        cur_val_loss = 0
        for j in range(len(xtest)):
            cur_val_loss += (y_pred[j]-ytest[j])**2
        if deg == 0:
            best_val_loss = cur_val_loss
            best_deg = deg
            best_coeff = cur_coeff
        else:
            if cur_val_loss < best_val_loss:
                best_val_loss = cur_val_loss
                best_deg = deg
                best_coeff = cur_coeff
    return best_deg,best_val_loss,best_coeff

# disable RankWarning from np.polyfit using the following line of code
warnings.simplefilter('ignore', np.RankWarning)
# disable ConvergenceWarning from ARIMA using the following line of code
warnings.simplefilter('ignore', ConvergenceWarning)
# disable UserWarning from ARIMA using the following line of code
warnings.simplefilter('ignore', category=UserWarning)

header = ["County number", "Lowest Polynomial Validation Loss", "Lowest ARIMA Validation Loss", "Most Effective Method"]
table_info = []
for city_index in range(9): # for each of the 9 cities
    xtrain = Y_training
    xtest = Y_test
    ytrain = x_training[city_index]
    ytest = x_test[city_index]
    # calculate best parameter (deg) in polynomial regression
    poly_best_deg,poly_lowest_val_loss,poly_coeff = validation_loss(xtrain,ytrain,xtest,ytest)
    # calculate best parameters (p,q,r) in ARIMA
    arima_lowest_val_loss = np.inf # initially set lowest validation loss to +infinity
    arima_best_params = []
    for p in range(3):
        for d in range(3):
            for q in range(3):
                cur_arima_val_loss = ARIMA_val_loss(ytrain,ytest,p,d,q)
                if cur_arima_val_loss < arima_lowest_val_loss:
                    arima_lowest_val_loss = cur_arima_val_loss
                    arima_best_params = (p,d,q)
    if arima_lowest_val_loss < poly_lowest_val_loss:
        lowest_val_loss = 'ARIMA'
        
    else:
        lowest_val_loss = 'Polynomial Regression'
    table_row = ([city_index+1, poly_lowest_val_loss, arima_lowest_val_loss, lowest_val_loss])
    table_info.append(table_row)
    print('City number '+str(city_index+1))
    print('Best polynomial degree:')
    print(poly_best_deg)
    print('Best polynomial validation loss:')
    print(poly_lowest_val_loss)
    print('Best ARIMA parameters:')
    print(arima_best_params)
    print('Best ARIMA validation loss:')
    print(arima_lowest_val_loss)

print(tabulate(table_info, headers=header, tablefmt='fancy_grid'))

City number 1
Best polynomial degree:
0
Best polynomial validation loss:
0.0201818201320551
Best ARIMA parameters:
(0, 0, 0)
Best ARIMA validation loss:
0.020185155014345476
City number 2
Best polynomial degree:
0
Best polynomial validation loss:
0.002962581692573554
Best ARIMA parameters:
(0, 2, 0)
Best ARIMA validation loss:
0.0019181577934050534
City number 3
Best polynomial degree:
0
Best polynomial validation loss:
0.4713731065451134
Best ARIMA parameters:
(2, 0, 1)
Best ARIMA validation loss:
0.18159342408303575
City number 4
Best polynomial degree:
1
Best polynomial validation loss:
0.011541154465973572
Best ARIMA parameters:
(1, 2, 0)
Best ARIMA validation loss:
0.06473930680848956
City number 5
Best polynomial degree:
2
Best polynomial validation loss:
0.0076897455961454685
Best ARIMA parameters:
(0, 0, 0)
Best ARIMA validation loss:
0.268508025051864
City number 6
Best polynomial degree:
3
Best polynomial validation loss:
0.6101282200992125
Best ARIMA parameters:
(0, 0, 0)
Be